In [1]:
import os
import platform
import logging

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import tqdm
import cv2
import numpy as np
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.fr_detector import FRDetector
from street_signs_identify.detector.craft_detector import CRAFTDetector

In [2]:
from street_signs_identify.data_type.detected_image import DetectedImage

class TowStepDetection_for_TEST:
    def __init__(self, fr_detector:FRDetector, craft_detector:CRAFTDetector):
        self._fr_detector = fr_detector
        self._craft_detector = craft_detector

    def _step1(self, gt_image: DetectedImage) -> DetectedImage:
        fr_image = self._fr_detector(gt_image.image)
        if fr_image is None:
            return None
        
        overlapping_info = co.check_overlapping_area(gt_image.info, fr_image.info)
        if overlapping_info is None:
            return None
        
        fr_image = fr_image.keeps_index(overlapping_info["index_2nd"])
        return fr_image

    def _step2(self, fr_image: DetectedImage):
        new_df = pd.DataFrame(columns=["label", "score", "x0", "y0", "x1", "y1", "frcnn_label", "frcnn_score"])
        
        for idx in fr_image.info.index:
            fr_instance = fr_image[idx]
            craft_image = self._craft_detector(fr_instance.image)
            if craft_image is None:
                continue
        
            x0_ori, y0_ori, _, _ = fr_instance.bbox
            craft_info = craft_image.info
            craft_info["x0"] += x0_ori
            craft_info["x1"] += x0_ori
            craft_info["y0"] += y0_ori
            craft_info["y1"] += y0_ori
            craft_info["frcnn_label"] = fr_instance.info
            craft_info["frcnn_score"] = fr_instance.score
            new_df = pd.concat([new_df,craft_info], ignore_index=True)
        
        return DetectedImage(fr_image._ref_image, new_df)
    
    def detect(self, gt_image: DetectedImage) -> DetectedImage:
        fr_image = self._step1(gt_image)
        if fr_image is None:
            return None
        craft_image = self._step2(fr_image)
        if craft_image.info.empty:
            return None
        return craft_image

In [3]:
LOGGER = logging.getLogger(__name__)

In [4]:
if platform.system() == "Linux":
    model_path = "./street_signs_identify/detector/saved_models/frcnn.pth"
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "frcnn_craft_cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")

else:
#     folder_name = "train"
#     model_path = r"C:\Users\MAGIC\Desktop\model\frcnn\best_map_frcnn_FrzModel.pth"
#     data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
#     output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc\\{folder_name}'
#     output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc_gt\\{folder_name}'
#     output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc_filter\\{folder_name}'
#     output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xfc_fast_craft\\{folder_name}'
    
    # demo
    model_path = r"C:\Users\MAGIC\Desktop\model\frcnn\best_map_frcnn_FrzModel.pth"
    data_path = r'C:\Users\MAGIC\Ching\survey\plan\rawdata_all\7types_v2\demo'
    output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc'
    output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc_gt'
    output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc_filter'
    output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\demo\\xfc_fast_craft'



os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True) 
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [5]:
frcnn_detector = FRDetector(model_ckpt=model_path, parse_label=True)
craft_detector = CRAFTDetector()
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232747579_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232747926_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232748282_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232748649_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232749037_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232749458_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232749923_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232750457_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232751091_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232751944_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Penghu/220720_232753091_Camera_0.jpg
Ignore data whose file is not exist: 
./dataset/2Pengh

In [6]:
two_step_detector = TowStepDetection_for_TEST(frcnn_detector, craft_detector)

In [7]:
x_image_loader._info.head()

,raw_filename,filename,category,box,label,polygon,x0,y0,x1,y1
0,220720_225837855_Camera_0.jpg,NaN,3,"[(1853, 861), (1861, 927), (1996, 951), (1987,...",永順\nYongshu,"POLYGON ((1853 861, 1861 927, 1996 951, 1987 8...",1853,861,1996,951
1,220720_225953347_Camera_0.jpg,NaN,3,"[(1469, 789), (1473, 842), (1616, 848), (1608,...",永順街\nYingshun St.,"POLYGON ((1469 789, 1473 842, 1616 848, 1608 7...",1469,789,1616,848
2,220720_225953866_Camera_0.jpg,NaN,3,"[(1565, 701), (1574, 766), (1738, 779), (1727,...",永順街\nYongshun St.,"POLYGON ((1565 701, 1574 766, 1738 779, 1727 7...",1565,701,1738,779
3,220720_225954302_Camera_0.jpg,NaN,3,"[(1706, 572), (1722, 647), (1907, 675), (1889,...",永順街\nYongshun St.,"POLYGON ((1706 572, 1722 647, 1907 675, 1889 6...",1706,572,1907,675
4,220720_230048513_Camera_0.jpg,NaN,3,"[(1615, 739), (1618, 799), (1762, 812), (1760,...",同義和路\nTonghe Rd.,"POLYGON ((1615 739, 1618 799, 1762 812, 1760 7...",1615,739,1762,812


In [8]:
import numpy as np

global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "gt_index", "CRAFT2GT_IOU", "frcnn_label"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    gt_image = x_image_loader[image_name]


    detected_image = two_step_detector.detect(gt_image)
    if detected_image is None:
        LOGGER.warning("Frcnn detection failure: %s", image_name)
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        LOGGER.warning("Detected instance and GT are all disjoint: %s", image_name)
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    filtered_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = filtered_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))


    for row_idx, row in overlapping_info.iterrows():
#         if row["IOU"] < .5: # skip frcnn iou < .5
#             continue

        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue

        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
        label_df.loc[global_index, "CRAFT2GT_IOU"] = row["IOU"]
        label_df.loc[global_index, "gt_index"] = int(gt_index)
        label_df.loc[global_index, "frcnn_label"] = detected_image.info.loc[detected_index, "frcnn_label"] # frcnn_label

        crapped_img = detected_instance.image


        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
        global_index+=1

label_df.to_csv(os.path.join(output_folder, "labels.csv"), encoding = 'big5', index=False)
print('all is done.')

100%|██████████| 162/162 [01:30<00:00,  1.80it/s]

all is done.


In [9]:
label_df

,filename,category,words,src_file,gt_index,CRAFT2GT_IOU,frcnn_label
0,0.jpg,3,永順\nYongshu,220720_225837855_Camera_0.jpg,0,0.25949,green
1,1.jpg,3,永順街\nYingshun St.,220720_225953347_Camera_0.jpg,0,0.497149,green
2,2.jpg,3,永順街\nYingshun St.,220720_225953347_Camera_0.jpg,0,0.090359,green
3,3.jpg,3,永順街\nYongshun St.,220720_225953866_Camera_0.jpg,0,0.43245,green
4,4.jpg,3,永順街\nYongshun St.,220720_225954302_Camera_0.jpg,0,0.036907,green
...,...,...,...,...,...,...,...
456,456.jpg,3,樹德路\nShude Rd.,220720_234137732_Camera_0.jpg,0,0.337393,green
457,457.jpg,3,樹德路\nShude Rd.,220720_234137732_Camera_0.jpg,0,0.133673,green
458,458.jpg,3,樹德路\nShude Rd.,220720_234137732_Camera_0.jpg,0,0.052905,green
459,459.jpg,4,公務專用\n禁止進入,220720_234353203_Camera_0.jpg,0,0.475975,red
